# Clasificador BackPropagattion



Importar las librerías necesarias

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.metrics import Accuracy
from keras.preprocessing import image
import numpy as np
from tensorflow.keras.preprocessing import image
import numpy as np



### Preprocesamiento de las imagenes

1. **ImageDataGenerator:**
   - `train_datagen` y `test_datagen` son instancias de `ImageDataGenerator`. Estas realizan aumentación y normalización de imágenes durante el entrenamiento y la prueba.

2. **Flujo de Datos de Entrenamiento y Prueba:**
   - `flow_from_directory` crea generadores de flujo de datos desde directorios de imágenes.
   - `target_size=(64, 64)` ajusta el tamaño de todas las imágenes a 64x64 píxeles.
   - `batch_size=32` define el tamaño del lote para el entrenamiento y la prueba.
   - `class_mode='categorical'` indica que se trata de un problema de clasificación multiclase.

3. **Directorios de Imágenes:**
   - `imagenes` y `imagenes_prueba` son los directorios de donde se obtienen las imágenes para entrenamiento y prueba, respectivamente.

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory(
    'imagenes',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    'imagenes_prueba',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)


Found 2840 images belonging to 5 classes.
Found 387 images belonging to 5 classes.


### Construccion del modelo


1. **Capa Convolucional:**
   - `Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3))`: La primera capa convolucional con 32 filtros, cada uno de tamaño 3x3, y activación ReLU. La entrada es de forma (64, 64, 3), indicando imágenes de 64x64 píxeles con 3 canales de color (RGB).

2. **Capa de Pooling:**
   - `MaxPooling2D(pool_size=(2, 2))`: Capa de reducción de dimensionalidad mediante max pooling con una ventana de 2x2. Esto ayuda a conservar las características más importantes.

3. **Aplanamiento:**
   - `Flatten()`: Aplana la salida de la capa anterior para prepararla para las capas totalmente conectadas.

4. **Capa Densa (Totalmente Conectada):**
   - `Dense(units=128, activation='relu')`: Capa densa con 128 unidades y activación ReLU.

5. **Capa de Salida:**
   - `Dense(units=5, activation='softmax')`: Capa de salida con 5 unidades y activación softmax, adecuada para problemas de clasificación multiclase. Las 5 clases representan autobús, avión, bicicleta, bote y sofá.
 

In [3]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=5, activation='softmax'))  # 5 clases: autobus, avion, bicicletas, botes, sofa


### Compilacion del modelo

1. **Optimizador Adam:**
   - `Adam(learning_rate=0.0001)`: Se utiliza el optimizador Adam con una tasa de aprendizaje de 0.0001. Adam es un optimizador popular en problemas de aprendizaje profundo.

2. **Función de Pérdida Categórica:**
   - `loss='categorical_crossentropy'`: La función de pérdida utilizada para entrenar el modelo en problemas de clasificación multiclase.

3. **Métricas de Evaluación:**
   - `metrics=[Accuracy()]`: Durante el entrenamiento, se mide la precisión del modelo. La métrica de precisión evalúa cuántas predicciones del modelo coinciden con las etiquetas reales.

In [4]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=[Accuracy()])

### Entrenamiento del modelo

In [5]:
history = model.fit(
    train_set,
    steps_per_epoch=len(train_set),
    epochs=8,
    validation_data=test_set,
    validation_steps=len(test_set)
)


Epoch 1/8
89/89 [==============================] - 20s 222ms/step - loss: 1.3324 - accuracy: 0.0000e+00 - val_loss: 1.1349 - val_accuracy: 0.0000e+00
Epoch 2/8
89/89 [==============================] - 7s 73ms/step - loss: 1.0800 - accuracy: 0.0000e+00 - val_loss: 1.0933 - val_accuracy: 0.0000e+00
Epoch 3/8
89/89 [==============================] - 7s 74ms/step - loss: 0.9633 - accuracy: 0.0000e+00 - val_loss: 0.9429 - val_accuracy: 0.0000e+00
Epoch 4/8
89/89 [==============================] - 7s 74ms/step - loss: 0.8681 - accuracy: 0.0000e+00 - val_loss: 0.9669 - val_accuracy: 0.0000e+00
Epoch 5/8
89/89 [==============================] - 6s 72ms/step - loss: 0.8103 - accuracy: 0.0000e+00 - val_loss: 0.8119 - val_accuracy: 0.0000e+00
Epoch 6/8
89/89 [==============================] - 7s 74ms/step - loss: 0.7404 - accuracy: 0.0000e+00 - val_loss: 0.7879 - val_accuracy: 0.0000e+00
Epoch 7/8
89/89 [==============================] - 7s 73ms/step - loss: 0.6830 - accuracy: 0.0000e+00 - val_lo

In [6]:
# Evaluar el modelo en el conjunto de prueba
eval_result = model.evaluate(test_set)
print(f"Test loss: {eval_result[0]}, Test accuracy: {eval_result[1]}")

# Si deseas ver el historial de entrenamiento para observar cómo evolucionan las métricas
print(history.history)


13/13 [==============================] - 1s 51ms/step - loss: 0.6930 - accuracy: 0.0000e+00
Test loss: 0.692962646484375, Test accuracy: 0.0
{'loss': [1.3324187994003296, 1.0799846649169922, 0.9633239507675171, 0.868134081363678, 0.8103399276733398, 0.7403931617736816, 0.6830040812492371, 0.6381073594093323], 'accuracy': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'val_loss': [1.1348859071731567, 1.0932586193084717, 0.9428659081459045, 0.9669401049613953, 0.8118992447853088, 0.7878959774971008, 0.7230649590492249, 0.6929625868797302], 'val_accuracy': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}


In [7]:
img_path = "imagenes_prueba/autobus/2008_004968.jpg"

1. **Cargar y Preprocesar la Imagen:**
   - `image.load_img(img_path, target_size=(64, 64))`: Carga la imagen desde la ruta especificada y la redimensiona a 64x64 píxeles.
   - `image.img_to_array(img)`: Convierte la imagen a un array NumPy.
   - `np.expand_dims(img_array, axis=0)`: Expande las dimensiones del array para que coincida con las expectativas del modelo.

2. **Normalización:**
   - `img_batch /= 255.`: Normaliza los píxeles de la imagen dividiendo por 255, asegurando que estén en el rango [0, 1].

3. **Predicción:**
   - `model.predict(img_batch)`: Utiliza el modelo preentrenado para hacer una predicción sobre la imagen.
   - `np.argmax(prediction)`: Obtiene el índice de la clase predicha con la mayor probabilidad.

4. **Traducción de la Etiqueta Numérica:**
   - `labels = train_set.class_indices`: Obtiene un diccionario de índices de clases desde el conjunto de entrenamiento.
   - `labels = dict((v, k) for k, v in labels.items())`: Invierte el diccionario para mapear índices a etiquetas.
   - `labels[prediction_label]`: Obtiene la etiqueta predicha basada en el índice devuelto por la predicción.

5. **Impresión del Resultado:**
   - `print(f"El modelo predice que la imagen es un(a) {predicted_label}")`: Imprime la etiqueta predicha por el modelo.


In [8]:
img = image.load_img(img_path, target_size=(64, 64))

# Convertir la imagen a un array y expandir las dimensiones para que se ajuste al modelo
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)

 
img_batch /= 255.

# Hacer la predicción
prediction = model.predict(img_batch)
prediction_label = np.argmax(prediction)

# Traducir la etiqueta numérica a una palabra clave de acuerdo a tu conjunto de datos
labels = train_set.class_indices
labels = dict((v, k) for k, v in labels.items())
predicted_label = labels[prediction_label]

print(f"El modelo predice que la imagen es un(a) {predicted_label}")


1/1 [==============================] - 0s 64ms/step
El modelo predice que la imagen es un(a) autobus


### Metricas

In [9]:
img = image.load_img(img_path, target_size=(64, 64))

# Convertir la imagen a un array y expandir las dimensiones para que se ajuste al modelo
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

In [10]:
predictions = model.predict(img_array)

1/1 [==============================] - 0s 18ms/step


In [11]:
predicted_index = np.argmax(predictions[0])
predicted_label = labels[predicted_index]  # Suponiendo que 'labels' es una lista de las etiquetas de tus clases
predicted_probability = predictions[0][predicted_index]

In [12]:
print(f"Etiqueta predicha: {predicted_label}, Probabilidad: {predicted_probability * 100:.2f}%")


Etiqueta predicha: autobus, Probabilidad: 100.00%
